In [1]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
from huggingface_hub import login
login()

In [3]:
import torch
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch.nn.functional as F
import numpy as np
import csv
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",  output_attentions=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Lin

In [5]:
input_text = "Once upon a time, in a quaint little village nestled between rolling hills and lush forests, there existed a village named Green Valley, was home to villagers who lived peaceful and harmonious lives. A clear stream flowed through the village, nourishing the fields and orchards on its banks. Every morning, the villagers would rise early to begin their day’s work. The men worked in the fields, the women wove and cooked at home, and the children studied at the village school.In the village lived an elderly man named Grandpa Li, the village sage. Villagers would seek his advice whenever they faced difficulties. Grandpa Li was not only knowledgeable but also a great storyteller. Every evening, villagers would gather in front of his house to listen to his tales of ancient legends and magical adventures.One day, a stranger arrived in the village. He introduced himself as a traveler named Amin, carrying a mysterious package filled with strange items. The villagers, curious about Amin, gathered around to ask about his origins and experiences. Amin smiled and told them he came from a distant land in the East, having traveled to many places and witnessed many wonders.Amin decided to stay in the village, leading the villagers on explorations of the surrounding forests and hills, uncovering hidden treasures and secrets. Under his guidance, the villagers discovered rare flowers, plants, and animals they had never seen before. Amin also taught them new skills and knowledge, enriching their lives.However, Amin’s arrival was not by chance. He was guided by an ancient prophecy to Green Valley, seeking a legendary mystical power said to be hidden in the village. This power could only be found by those who were pure of heart and brave. Amin believed the villagers of Green Valley were the ones he was looking for.Amin decided to share this secret with the villagers. He gathered them under the big tree in the village and told them about the ancient prophecy. The villagers were both surprised and excited, eager to help Amin find this mystical power. Thus, under Amin’s leadership, they embarked on a journey filled with adventures and challenges.They traversed dense forests, climbed steep mountains, and overcame numerous obstacles. Eventually, they found the mystical power in a hidden cave. This power not only made the village more prosperous but also purified and elevated the villagers’ spirits.From then on, Green Valley became even more beautiful and harmonious, and the villagers grew more united and loving. Amin continued his journey, seeking more miracles and adventures. The story of Green Valley spread far and wide, told by Grandpa Li to all who would listen."

tokenized_input = tokenizer(input_text, return_tensors="pt", truncation=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [6]:
# Perform inference and obtain attention weights for all layers
outputs = model(**tokenized_input)
all_attention_weights = outputs.attentions  # This is a tuple of tensors
# Extract hidden states
all_hidden_states = outputs.hidden_states

In [8]:
for i, hidden_state in enumerate(all_hidden_states):
    print(f"Layer {i} hidden state shape: {hidden_state.shape}")

TypeError: 'NoneType' object is not iterable

In [8]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'][0])

# 遍历每一层的注意力权重
for layer_idx, attention_weights in enumerate(all_attention_weights):
    # 去掉batch维度
    attention_weights = attention_weights.squeeze(0).detach().numpy()
    
    # 遍历每个head
    num_heads = attention_weights.shape[0]
    for head_idx in range(num_heads):
        # 为每个层和head组合创建一个CSV文件
        file_name = f'attention_weights_layer_{layer_idx + 1}_head_{head_idx + 1}.csv'
        with open(file_name, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            
            # 写入CSV的表头（横轴tokens）
            writer.writerow([''] + tokens)
            
            # 遍历每个token对
            for i, token1 in enumerate(tokens):
                # 获取当前行对应的注意力分数
                row_attention_scores = attention_weights[head_idx, i, :]
                
                # 写入当前行的token和对应的注意力分数
                writer.writerow([token1] + list(row_attention_scores))

print("注意力分数矩阵已按层和head保存至不同的CSV文件中")

注意力分数矩阵已按层和head保存至不同的CSV文件中


In [7]:
batch_idx = 0
# 遍历所有相邻层进行比较
for layer_idx in range(num_layers - 1):
    # 获取当前层和下一层的 hidden states
    layer_hidden_states = all_hidden_states[layer_idx][batch_idx] 
    
    # 对于每个头，计算相邻层之间的余弦相似度

    # Normalize hidden states
    norm_layer_hidden_states = layer_hidden_states / layer_hidden_states.norm(dim=-1, keepdim=True)

    # Calculate cosine similarity between tokens across layers
    cosine_similarity = torch.mm(norm_layer_hidden_states, norm_layer_hidden_states.T)

    # Create a mask for the lower triangle of the matrix
    num_tokens = cosine_similarity.size(0)
    mask = np.tril(np.ones((num_tokens, num_tokens)), k=0)

    # Apply the mask to the cosine similarity matrix
    masked_similarity = cosine_similarity.detach().numpy() * mask

    # 将相似度矩阵保存到CSV文件
    csv_filename = f"HS_cos_sim_layer_{layer_idx}.csv"
    with open(csv_filename, mode='w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        for row in masked_similarity:
            csv_writer.writerow(row)

print("所有相邻层不同注意力头的余弦相似度矩阵已保存到CSV文件。")


所有相邻层不同注意力头的余弦相似度矩阵已保存到CSV文件。
